In [ ]:
import pandas as pd

In [ ]:
DATA_PATH = '/kaggle/input/lish-moa/'

In [ ]:
train_features = pd.read_csv(DATA_PATH + 'train_features.csv', index_col=False)
train_targets = pd.read_csv(DATA_PATH + 'train_targets_scored.csv', index_col=False)
train_targets_ns = pd.read_csv(DATA_PATH + 'train_targets_nonscored.csv', index_col=False)
test_features = pd.read_csv(DATA_PATH + 'test_features.csv', index_col=False)

submission = pd.read_csv(DATA_PATH + 'sample_submission.csv', index_col=False)

In [ ]:
# Reset values in sample submission
submission.loc[:, submission.columns != 'sig_id'] = 0

# Model

## Prepare environment

In [ ]:
import tensorflow as tf
from sklearn.metrics import log_loss

In [ ]:
device_name = tf.test.gpu_device_name()

if "GPU" not in device_name:
    print("GPU device not found, using CPU")
else:
    print('Found GPU at: {}'.format(device_name))

## Prepare data

In [ ]:
# Make a copy and start working on it
std_train_features = train_features.copy()
std_test_features = test_features.copy()

### Top features

In [ ]:
top_feats = [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
             30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52,
             53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74,
             75, 76, 77, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97,
             98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
             116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
             137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 150, 151, 152, 153, 154, 155,
             156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173,
             174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191,
             192, 193, 194, 195, 196, 198, 199, 200, 201, 203, 204, 205, 206, 207, 209, 210, 211, 212,
             213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 224, 225, 226, 227, 228, 229, 230, 231,
             232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 243, 244, 245, 246, 247, 248, 249, 250,
             251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268,
             269, 270, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287,
             288, 289, 290, 291, 292, 293, 295, 296, 297, 299, 301, 302, 303, 304, 305, 306, 307, 308,
             309, 310, 311, 312, 313, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327,
             328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345,
             346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363,
             364, 365, 366, 367, 368, 369, 370, 371, 372, 375, 376, 377, 378, 379, 380, 381, 382, 383,
             384, 385, 386, 387, 388, 389, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402,
             403, 404, 405, 406, 407, 408, 409, 410, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421,
             422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 435, 436, 437, 438, 439, 440,
             441, 443, 444, 445, 446, 447, 448, 449, 450, 451, 454, 455, 457, 458, 459, 460, 461, 462,
             463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480,
             482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 499, 501,
             502, 503, 504, 506, 507, 508, 510, 511, 512, 513, 514, 515, 516, 519, 520, 521, 522, 523,
             524, 525, 526, 527, 528, 529, 531, 532, 533, 535, 536, 537, 539, 540, 541, 542, 543, 544,
             545, 546, 547, 548, 550, 551, 552, 553, 555, 558, 560, 561, 562, 563, 566, 567, 568, 569,
             570, 571, 572, 573, 574, 575, 576, 578, 579, 581, 582, 583, 584, 585, 586, 587, 588, 589,
             590, 591, 592, 593, 594, 595, 596, 597, 598, 600, 601, 602, 603, 607, 608, 609, 610, 612,
             613, 614, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631,
             632, 633, 634, 635, 636, 637, 638, 639, 640, 642, 643, 644, 645, 646, 647, 648, 649, 650,
             651, 652, 653, 655, 656, 657, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670,
             671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688,
             689, 690, 692, 693, 694, 695, 696, 697, 698, 700, 701, 702, 703, 705, 706, 708, 709, 710,
             711, 712, 714, 715, 717, 718, 719, 721, 722, 724, 725, 726, 727, 728, 729, 730, 731, 732,
             733, 734, 735, 736, 738, 739, 740, 741, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752,
             753, 754, 755, 756, 757, 758, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771,
             772, 773, 774, 775, 776, 777, 778, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790,
             791, 793, 794, 795, 796, 797, 798, 799, 801, 802, 803, 804, 805, 806, 807, 809, 810, 812,
             814, 815, 816, 817, 818, 819, 820, 822, 823, 824, 826, 827, 828, 829, 830, 831, 832, 833,
             835, 836, 838, 839, 840, 841, 842, 843, 846, 847, 848, 849, 851, 852, 853, 855, 856, 857,
             859, 860, 861, 862, 863, 865, 867, 868, 869, 870, 871, 872, 873, 874, 875]
print(len(top_feats))

In [ ]:
# Note that cp_type is removed later
def filter_top_features(std_train_features, std_test_features):
    top_feat_columns = std_train_features.columns[top_feats]
    std_train_features = std_train_features.loc[:, std_train_features.columns.isin(top_feat_columns)]
    std_test_features = std_test_features.loc[:, std_test_features.columns.isin(top_feat_columns)]
    return std_train_features, std_test_features

In [ ]:
std_train_features, std_test_features = filter_top_features(std_train_features, std_test_features)

In [ ]:
std_train_features

### PCA features

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# Get column names that we should preprocess later (before we add features with PCA)
preprocess_columns = std_train_features.columns.difference(['sig_id', 'cp_type', 'cp_time', 'cp_dose'])

In [ ]:
"""
# GENES
GENES = [col for col in std_train_features.columns if col.startswith('g-')]
n_comp = 50

data = pd.concat([pd.DataFrame(std_train_features[GENES]), pd.DataFrame(std_test_features[GENES])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[GENES]))
train2 = data2[:std_train_features.shape[0]]; test2 = data2[-std_test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
std_train_features = pd.concat((std_train_features, train2), axis=1)
std_test_features = pd.concat((std_test_features, test2), axis=1)
"""

# CELLS
CELLS = [col for col in std_train_features.columns if col.startswith('c-')]
n_comp = 15

data = pd.concat([pd.DataFrame(std_train_features[CELLS]), pd.DataFrame(std_test_features[CELLS])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[CELLS]))
train2 = data2[:std_train_features.shape[0]]; test2 = data2[-std_test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
std_train_features = pd.concat((std_train_features, train2), axis=1)
std_test_features = pd.concat((std_test_features, test2), axis=1)

### Normalization (not used)

In [ ]:
from sklearn import preprocessing
import numpy as np

In [ ]:
# Create a scaler with sklearn
scaler = preprocessing.StandardScaler().fit(std_train_features.loc[:, preprocess_columns])

#std_train_features.loc[:, preprocess_columns] = scaler.transform(std_train_features.loc[:, preprocess_columns])
#std_test_features.loc[:, preprocess_columns] = scaler.transform(std_test_features.loc[:, preprocess_columns])

### Categorical data

In [ ]:
# Transform cp_type column
std_train_features.loc[:, 'cp_type'] = std_train_features.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
std_test_features.loc[:, 'cp_type'] = std_test_features.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})

# Transform cp_time column
std_train_features.loc[:, 'cp_time'] = std_train_features.loc[:, 'cp_time'].map({24: 0, 48: 1, 72: 2})
std_test_features.loc[:, 'cp_time'] = std_test_features.loc[:, 'cp_time'].map({24: 0, 48: 1, 72: 2})

# Transform cp_dose column
std_train_features.loc[:, 'cp_dose'] = std_train_features.loc[:, 'cp_dose'].map({'D1': 3, 'D2': 4})
std_test_features.loc[:, 'cp_dose'] = std_test_features.loc[:, 'cp_dose'].map({'D1': 3, 'D2': 4})

## Define model and callbacks

In [ ]:
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
import tensorflow.keras.backend as K

from sklearn.model_selection import StratifiedKFold

In [ ]:
class ValLossDisplay(tf.keras.callbacks.Callback):
    def __init__(self, loss_index='val_loss'):
        self.best_loss = None
        self.loss_index = loss_index
        super().__init__()
    
    def on_epoch_end(self, epoch, logs=None):
        if self.best_loss is None or self.best_loss > logs[self.loss_index]:
            self.best_loss = logs[self.loss_index]
        print("\r",'Epoch: %d - Loss: %.6f (best: %.6f)' % (epoch, logs[self.loss_index], self.best_loss), ' '*5, end='')
    
    def on_train_end(self, logs=None):
        print("\r",'Loss: %.6f' % (self.best_loss), ' '*40)

In [ ]:
class SGDRScheduler(tf.keras.callbacks.Callback):
    '''Cosine annealing learning rate scheduler with periodic restarts.
    # Usage
        ```python
            schedule = SGDRScheduler(min_lr=1e-5,
                                     max_lr=1e-2,
                                     steps_per_epoch=np.ceil(epoch_size/batch_size),
                                     lr_decay=0.9,
                                     cycle_length=5,
                                     mult_factor=1.5)
            model.fit(X_train, Y_train, epochs=100, callbacks=[schedule])
        ```
    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`. 
        lr_decay: Reduce the max_lr after the completion of each cycle.
                  Ex. To reduce the max_lr by 20% after each cycle, set this value to 0.8.
        cycle_length: Initial number of epochs in a cycle.
        mult_factor: Scale epochs_to_restart after each full cycle completion.
    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: http://arxiv.org/abs/1608.03983
    '''
    def __init__(self,
                 min_lr,
                 max_lr,
                 steps_per_epoch,
                 lr_decay=1,
                 cycle_length=10,
                 mult_factor=2):

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.lr_decay = lr_decay

        self.batch_since_restart = 0
        self.next_restart = cycle_length

        self.steps_per_epoch = steps_per_epoch

        self.cycle_length = cycle_length
        self.mult_factor = mult_factor

        self.history = {}

    def clr(self):
        '''Calculate the learning rate.'''
        fraction_to_restart = self.batch_since_restart / (self.steps_per_epoch * self.cycle_length)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(fraction_to_restart * np.pi))
        return lr

    def on_train_begin(self, logs={}):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.max_lr)

    def on_batch_end(self, batch, logs={}):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        self.batch_since_restart += 1
        K.set_value(self.model.optimizer.lr, self.clr())

    def on_epoch_end(self, epoch, logs={}):
        '''Check for end of current cycle, apply restarts when necessary.'''
        if epoch + 1 == self.next_restart:
            self.batch_since_restart = 0
            self.cycle_length = np.ceil(self.cycle_length * self.mult_factor)
            self.next_restart += self.cycle_length
            self.max_lr *= self.lr_decay
            self.best_weights = self.model.get_weights()

    def on_train_end(self, logs={}):
        '''Set weights to the values from the end of the most recent cycle for best performance.'''
        self.model.set_weights(self.best_weights)

In [ ]:
def build_model(n_features, n_targets):
    layers = [
        tf.keras.layers.Input(n_features),
        tf.keras.layers.Dense(512),
        tf.keras.layers.PReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(512),
        tf.keras.layers.PReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(n_targets, activation='sigmoid')
    ]

    optimizer = tfa.optimizers.AdamW(lr=1e-4, weight_decay=1e-5, clipvalue=700)
    
    model = tf.keras.Sequential(layers)
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy'
    )

    return model

In [ ]:
# For debug purposes, display summary of the model
n_features = len(top_feats) - 2
n_targets = len(list(train_targets.columns)) - 1
model = build_model(n_features, n_targets)
model.summary()

## Training

In [ ]:
# Values for training
EPOCHS = 50
BATCH_SIZE = 128
NUM_SPLITS = 7
EARLY_STOP = 8 # Patience value for early stop
REDUCE_LR = 3
RANDOM_SEEDS = range(7)

In [ ]:
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
def train_and_predict(std_train_features, train_targets, std_test_features, submission, oof_df):
    tf.random.set_seed(42)
    
    # Remove the ID column for targets/labels of model training
    X_train = std_train_features.loc[:, std_train_features.columns != 'sig_id']
    y_train = train_targets.loc[:, train_targets.columns != 'sig_id']
    X_test = std_test_features.loc[:, std_test_features.columns != 'sig_id']

    # Values for model creation
    n_features = X_train.shape[1]
    n_targets = y_train.shape[1]

    # Init vars
    oof_indices = std_train_features['sig_id']
    history = []

    # Average of several seeds
    for seed in RANDOM_SEEDS:

        # Apply k-folds
        kfold = MultilabelStratifiedKFold(n_splits=NUM_SPLITS, shuffle=True, random_state=seed)
    
        for fold, (train_index, valid_index) in enumerate(kfold.split(X_train, y_train)):
            print('** SEED %d - FOLD %d **' % (seed, fold + 1))

            X_kfold_train, X_kfold_valid = X_train.values[train_index], X_train.values[valid_index]
            y_kfold_train, y_kfold_valid = y_train.values[train_index], y_train.values[valid_index]

            # Callbacks for early stopping
            displayer = ValLossDisplay()
            
            epoch_size = len(X_train)
            eas = EarlyStopping(monitor='val_loss', patience=EARLY_STOP, min_delta=1e-5,
                                verbose=1, mode='min', baseline=None, restore_best_weights=True)
            lr_sched = SGDRScheduler(min_lr=1e-5,
                                     max_lr=1e-2,
                                     steps_per_epoch=np.ceil(epoch_size/BATCH_SIZE),
                                     lr_decay=0.85,
                                     mult_factor=1.5)
            
            checkpoint_path = '/checkpoint_%d_%d' % (seed, fold)
            m_ckpt = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss',
                                                        verbose=0, save_best_only=True, save_weights_only=True)
            eas = EarlyStopping(monitor='val_loss', patience=EARLY_STOP,
                                verbose=1, mode='min', baseline=None, restore_best_weights=True)
            callbacks = [displayer, lr_sched, m_ckpt, eas]

            # Create model
            model = build_model(n_features, n_targets)

            kfold_history = model.fit(
                X_kfold_train,
                y_kfold_train,
                epochs=EPOCHS,
                batch_size=BATCH_SIZE,
                validation_data=(X_kfold_valid, y_kfold_valid),
                verbose=0,
                callbacks=callbacks
            )

            history.append(min(kfold_history.history['val_loss']))

            # Restore best weights
            model.load_weights(checkpoint_path)

            # Stores validation data prediction at respective indices
            oof_df.loc[oof_df['sig_id'].isin(oof_indices[valid_index]), oof_df.columns != 'sig_id'] += model.predict(X_kfold_valid)

            # adds up test prediction per fold
            kfold_pred = model.predict(X_test.values)
            submission.loc[submission['sig_id'].isin(std_test_features['sig_id']), submission.columns != 'sig_id'] += kfold_pred
    
    overall_loss = sum(history) / len(history)
    print('Overall Val. Loss: %.6f' % overall_loss)
    
    submission.loc[submission['sig_id'].isin(std_test_features['sig_id']), submission.columns != 'sig_id'] /= (NUM_SPLITS * len(RANDOM_SEEDS))
    oof_df.loc[:, oof_df.columns != 'sig_id'] /= len(RANDOM_SEEDS)
    
    return submission, oof_df

In [ ]:
# Init dataframes
submission.loc[submission['sig_id'].isin(std_test_features['sig_id']), submission.columns != 'sig_id'] = 0
oof_df = train_targets.copy()
oof_df.loc[:, train_targets.columns != 'sig_id'] = 0

# Data to train model when cp_type=trt_cp
gr0_std_train_features = std_train_features[std_train_features['cp_type'] == 0].reset_index(drop=True)
gr0_std_test_features = std_test_features[std_test_features['cp_type'] == 0].reset_index(drop=True)
gr0_train_targets = train_targets.loc[train_targets['sig_id'].isin(gr0_std_train_features['sig_id'])].reset_index(drop=True)
del gr0_std_train_features['cp_type']
del gr0_std_test_features['cp_type']

# Training and prediction when cp_type=trt_cp
submission, oof_df = train_and_predict(gr0_std_train_features, gr0_train_targets, gr0_std_test_features, submission, oof_df)

# Fix values to zero when cp_type=ctl_vehicle
submission.loc[std_test_features['cp_type']==1, submission.columns != 'sig_id'] = 0
oof_df.loc[std_train_features['cp_type']==1, oof_df.columns != 'sig_id'] = 0

In [ ]:
def logloss_metric(y_true, y_pred):
    metrics = []
    for _target in y_true.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels=[0,1]))
    return np.mean(metrics)

In [ ]:
logloss_score = logloss_metric(
    train_targets.loc[:, train_targets.columns != 'sig_id'],
    oof_df.loc[:, oof_df.columns != 'sig_id']
)

print('OOF Metric: %.6f' % logloss_score)

# Submit predictions

In [ ]:
submission.to_csv('submission.csv', index=False)